# Ejercicio: análisis morfológico y aplicación a opiniones sobre películas

<img src="img/drama.png" style="width:400px;height:400px;">

En este ejercicio vamos a utilizar críticas escritas en [IMDB](http://www.imdb.com/) para tratar de extraer automáticamente la opinión expresada, positiva o negativa, de un texto. Para ello utilizamos algunas técnicas técnicas de análisis morfológico del texto.

El objetivo del ejercicio es construir un sistema que dado el texto en inglés de una crítica sea capaz de estimar si esa crítica expresa una opinión positiva o negativa. Empezaremos construyendo un clasificador de opinión sencillo, para ir introduciendo características cada vez más complicadas e ir mejorando nuestros resultados.

## Instrucciones

A lo largo de este cuaderno encontrarás celdas vacías que tendrás que rellenar con tu propio código. Sigue las instrucciones del cuaderno y presta especial atención a los siguientes iconos:

<table>
<tr><td width="80"><img src="img/question.png" style="width:auto;height:auto"></td><td style="text-align:left">Deberás responder a la pregunta indicada con el código o contestación que escribas en la celda inferior.</td></tr>
 <tr><td width="80"><img src="img/exclamation.png" style="width:auto;height:auto"></td><td style="text-align:left">Esto es una pista u observación que te puede ayudar a resolver la práctica.</td></tr>
 <tr><td width="80"><img src="img/pro.png" style="width:auto;height:auto"></td><td style="text-align:left">Este es un ejercicio avanzado y voluntario que puedes realizar si quieres profundar más sobre el tema. Te animamos a intentarlo para aprender más ¡Ánimo!</td></tr>
</table>

Para evitar problemas de compatibilidad y de paquetes no instalados, se recomienda ejecutar este notebook bajo uno de los [entornos recomendados de Text Mining](https://github.com/albarji/teaching-environments/tree/master/textmining).

Adicionalmente si necesitas consultar la ayuda de cualquier función python puedes colocar el cursor de escritura sobre el nombre de la misma y pulsar Mayúsculas+Shift para que aparezca un recuadro con sus detalles. Ten en cuenta que esto únicamente funciona en las celdas de código.

¡Adelante!

## Preliminares

En primer lugar vamos a fijar la semilla aleatoria para que los resultados sean reproducibles entre diferentes ejecuciones del notebook.

In [ ]:
import numpy as np
np.random.seed(12345)

## Carga y preparación de datos

Los datos que usaremos en esta práctica son un conjunto preparado de los datos empleados en el artículo

    Andrew L. Maas, Raymond E. Daly, Peter T. Pham, Dan Huang, Andrew Y. Ng, and Christopher Potts. (2011). Learning Word Vectors for Sentiment Analysis. The 49th Annual Meeting of the Association for Computational Linguistics (ACL 2011).
    
y consisten en críticas de películas escritas en la web en inglés IMDB. Se han tomado aquellas críticas con una puntuación mayor a 7 como **opiniones positivas**, mientras que aquellas con puntuación menor a 4 se han tomado como **opiniones negativas**.

Los datos están todos contenidos en el fichero *data/data.csv*, en formato CSV separado por tabuladores. El fichero contiene únicamente dos columnas, la primera de ellas indicando el tipo de opinión (1 = opinión positiva, 0 = opinión negativa) y la segunda de ellas el texto de la crítica.

<table>
 <tr>
  <tr><td width="80"><img src="img/question.png" style="width:auto;height:auto"></td><td style="text-align:left">
      Carga en un Dataframe de Pandas los datos del fichero <i>data/data.csv</i>. Analiza los primeros registros del Dataframe. ¿Parecen coherentes los valores de opinión con el texto?
  </td>
 </tr> 
</table>

In [ ]:
####### INSERT YOUR CODE HERE

<table>
 <tr>
  <tr><td width="80"><img src="img/pro.png" style="width:auto;height:auto"></td><td style="text-align:left">
    El fichero cargado arriba es una versión reducida del conjunto completo de datos. Si quieres optar por usar todos los datos para esta práctica puedes cargar el fichero <i>data/datafull.csv.gz</i>. Ten en cuenta que los tiempos de cálculo serán mucho mayores, aunque a cambio podrás conseguir mejores resultados de clasificación.
  </td>
 </tr> 
</table>

In [ ]:
####### INSERT YOUR CODE HERE

Ahora vamos a preparar dos listas de índices, que nos indiquen qué parte de los datos vamos a usar para entrenamiento y qué parte para test.

<table>
 <tr>
  <tr><td width="80"><img src="img/question.png" style="width:auto;height:auto"></td><td style="text-align:left">
    Genera dos listas, una conteniendo los índices de la primera mitad de las filas del DataFrame de datos (índices de train), y otra conteniendo los índices de la otra mitad (índices de test).
  </td>
 </tr> 
</table>

In [ ]:
####### INSERT YOUR CODE HERE

## Modelo inicial

Para poder valorar si las técnicas avanzadas que vamos a emplear aportan algo de utilidad a este problema, vamos a empezar con una solución muy sencilla basada en bag of words, estimando la precisión de clasificación que podemos obtener con ella y usando este valor como referencia.

<table>
 <tr>
  <tr><td width="80"><img src="img/question.png" style="width:auto;height:auto"></td><td style="text-align:left">
    Utilizando lo que has aprendido en la práctica anterior, construye un sistema de clasificación basado en unigramas de palabras que aprenda de los datos de entrenamiento, y calcula el error de estimación en test del mismo. Como modelo de clasificación utiliza una SVM lineal, con sus parámetros por defecto. No realices ningún proceso de búsqueda para optimizar los parámetros del modelo (tipo GridSearchCV).
  </td>
 </tr> 
</table>

In [ ]:
####### INSERT YOUR CODE HERE

<table>
 <tr>
  <tr><td width="80"><img src="img/exclamation.png" style="width:auto;height:auto"></td><td style="text-align:left">
    El nivel de precisión que has obtenido, ¿crees que sería adecuado para una aplicación real? ¿Piensas que puede mejorarse?
  </td>
 </tr> 
</table>

## Análisis morfosintático con spaCy

En entornos donde existe mucho texto expresado de forma natural lo habitual es que una palabra aparezca con diversas conjugaciones y formas, sin que el significado final del texto cambie demasiado (salvo matices que discutiremos más adelante). En estos casos un paso de preprocesamiento habitual es convertir las palabras a lemas, o eliminar categorías morfológicas que aportan poca información. Para esto es imprescindible realizar un **análisis morfosintáctico** del texto, lo cual podemos hacer fácilmente para diversos idiomas utilizando la librería **spaCy**.

In [ ]:
import spacy

spaCy utiliza modelos morfosintácticos específicos para cada idioma. Por defecto la librería no incluye ningún modelo, pero podemos instalarlo de manera sencilla con comandos a python. La siguiente línea ejecuta un comando de sistema para instalar el modelo de spaCy para el idioma inglés.

In [ ]:
!python -m spacy download en

<table>
 <tr>
  <tr><td width="80"><img src="img/exclamation.png" style="width:auto;height:auto"></td><td style="text-align:left">
    Si el comando anterior produce un error relacionado con la falta de permisos, deberás ejecutarlo desde una terminal de Anaconda lanzada con permisos de administrador.
  </td>
 </tr> 
</table>

Una vez obtenido el modelo, podemos cargarlo en memoria con

In [ ]:
nlp = spacy.load('en')

y analizar una frase de ejemplo de la siguiente manera

In [ ]:
frase = "The black cat sat peacefully on the mat."
doc = nlp(frase)

**doc** es ahora una versión de la frase que contiene toda la información morfológica y sintáctica extraída por el analizador. Podemos iterar sobre cada uno de los tokens de la frase de la siguiente forma

In [ ]:
for token in doc:
    print("Token:", token)

Igualmente podemos acceder a cada uno de los tokens por su posición en la frase

In [ ]:
print(doc[2])

Pero lo más interesante son los diferentes campos con información extra que contiene cada token. Campos como
* *texto*: texto original
* *lemma_*: lema
* *pos_*: Part of Speech (categoría morfológica) simple
* *tag_*: categoría morfológica detallada
* *shape_*: patrón de mayúsculas/minúsculas
* *is_alpha*: si el token se componente de caracteres alfabéticos
* *is_stop*: si el token ha sido detectado como una stopword
* *head*: token padre en el árbol de dependencia
* *dep_*: relación sintáctica con el token padre
* etc...

Por ejemplo, vamos a imprimir toda esta información para el primer token de la frase

In [ ]:
token = doc[0]
print("Texto:", token.text)
print("Lema:", token.lemma_)
print("POS:", token.pos_)
print("Tag:", token.tag_)
print("Forma:", token.shape_)
print("Es alpha:", token.is_alpha)
print("Es stopword:", token.is_stop)
print("Token padre:", token.head)
print("Relación sintáctica:", token.dep_)

Podemos hacer esto con toda la frase y mostrarlo como una tabla (DataFrame) para mayor claridad

In [ ]:
pd.DataFrame(
    columns=["token", "lema", "POS", "tag", "shap", "isalpha", "isstop", "padre", "dep"],
    data=[[token.text, token.lemma_, token.pos_, token.tag_,
          token.shape_, token.is_alpha, token.is_stop, token.head, token.dep_]
          for token in doc]
)

También podemos visualizar el árbol sintático de dependencias usando la utilidad **displaCy**

In [ ]:
from spacy import displacy

displacy.render(doc, style='dep', jupyter=True)

Como hemos visto, el análisis morfosintáctico de spaCy nos proporciona mucha información, pero también puede ser costoso de realizar cuando tenemos gran cantidad de textos. Si no necesitamos de todos los componentes del análisis, podemos acelerar el tiempo de cálculo desactivando algunos elementos del proceso. Por ejemplo, cargando de nuevo el modelo de la siguiente forma

In [ ]:
nlpfast = spacy.load('en', disable=['ner', 'parser'])

Con esto tenemos un analizador morfosintáctico que no realiza detección de entidades (`ner`) ni análisis del árbol sintáctico de dependencias (`parser`), pero que a cambio ejecuta a mayor velocidad, como podemos comprobar en las siguientes dos celdas.

In [ ]:
%%time
for _ in range(10):
    nlp("The black cat sat peacefully on the mat.")

In [ ]:
%%time
for _ in range(10):
    nlpfast("The black cat sat peacefully on the mat.")

Visto el funcionamiento de spaCy, vamos a pasar a ejecutar el análisis morfosintáctico para cada texto de nuestros datos.

<table>
 <tr>
  <tr><td width="80"><img src="img/question.png" style="width:auto;height:auto"></td><td style="text-align:left">
    Crea una nueva columna en el DataFrame de datos que contenga una versión analizada con spaCy del texto correspondiente. Es suficiente con que apliques el objeto <b>nlp</b> a cada texto y guardes el resultado.
  </td>
 </tr> 
</table>

In [ ]:
####### INSERT YOUR CODE HERE

## Filtrado por morfología

Vamos a sacar partido a la información morfológica que nos proporciona spaCy para mejorar el modelo predictivo. Para ello realizaremos dos operaciones:

* Filtrar los textos para quedarnos solo con aquellas palabras de las categorías morfológicas con más carga de emoción.
* Filtrar los textos para no incluir palabras stopwords.
* Sustituir cada token por su lema, para así reducir el tamaño del vocabulario y simplificar el problema.

<table>
 <tr>
  <tr><td width="80"><img src="img/question.png" style="width:auto;height:auto"></td><td style="text-align:left">
    Crea una nueva columna en el DataFrame de datos que contenga una versión modificado del texto con únicamente los lemas de aquellos tokens cuyas etiquetas POS sean de clase <b>nombre</b>, <b>verbo</b>, <b>adjetivo</b> o <b>adverbio</b>.
  </td>
 </tr> 
</table>

In [ ]:
####### INSERT YOUR CODE HERE

<table>
 <tr>
  <tr><td width="80"><img src="img/question.png" style="width:auto;height:auto"></td><td style="text-align:left">
    Repite los pasos que realizaste en el caso del modelo inicial (al inicio de esta práctica) para construir un nuevo modelo, esta vez basado en los textos que has preparados en lugar de los textos originales. Mide el nivel de score sobre el conjunto de test, ¿has conseguido alguna mejora en precisión? ¿Y en tiempos de entrenamiento?
  </td>
 </tr> 
</table>

In [ ]:
####### INSERT YOUR CODE HERE